# Preprocessing
В этом ноутбуке будет некоторое количество фича-инженеринга и преобразования сырых данных.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(42)

## Winrates

Так уж MOBA-игры устроены, что в них всё немного нечестно. Разные герои сильны по-разному. Давайте для каждого героя посчитаем его винрейт и сделаем табличку с этими данными. В финальную таблицу, по которой мы и будем делать наши предикты, будут включены именно винрейты героев, потому что иначе мы получим категориальный признак со слишком большим количеством значений.

In [2]:
players_df = pd.read_csv('data/players.csv')

players_df.columns

Index(['match_id', 'account_id', 'hero_id', 'player_slot', 'gold',
       'gold_spent', 'gold_per_min', 'xp_per_min', 'kills', 'deaths',
       'assists', 'denies', 'last_hits', 'stuns', 'hero_damage',
       'hero_healing', 'tower_damage', 'item_0', 'item_1', 'item_2', 'item_3',
       'item_4', 'item_5', 'level', 'leaver_status', 'xp_hero', 'xp_creep',
       'xp_roshan', 'xp_other', 'gold_other', 'gold_death', 'gold_buyback',
       'gold_abandon', 'gold_sell', 'gold_destroying_structure',
       'gold_killing_heros', 'gold_killing_creeps', 'gold_killing_roshan',
       'gold_killing_couriers', 'unit_order_none',
       'unit_order_move_to_position', 'unit_order_move_to_target',
       'unit_order_attack_move', 'unit_order_attack_target',
       'unit_order_cast_position', 'unit_order_cast_target',
       'unit_order_cast_target_tree', 'unit_order_cast_no_target',
       'unit_order_cast_toggle', 'unit_order_hold_position',
       'unit_order_train_ability', 'unit_order_drop_item',


In [3]:
match_df = pd.read_csv('data/match.csv')

match_df.columns

Index(['match_id', 'start_time', 'duration', 'tower_status_radiant',
       'tower_status_dire', 'barracks_status_dire', 'barracks_status_radiant',
       'first_blood_time', 'game_mode', 'radiant_win', 'negative_votes',
       'positive_votes', 'cluster'],
      dtype='object')

In [4]:
wins = {}
games = {}

radiant = [0, 1, 2, 3, 4]
dire = [128, 129, 130, 131, 132]

heroes = [dict() for _ in range(match_df.shape[0])]

for _, row in players_df.iterrows():
    hero = row['hero_id']
    mid = row['match_id']
    
    if hero not in games.keys():
        games[hero] = 0
        wins[hero] = 0
    
    games[hero] += 1
    
    if row['player_slot'] in radiant and match_df['radiant_win'][mid]:
        wins[hero] += 1
    
    if row['player_slot'] in dire and not match_df['radiant_win'][mid]:
        wins[hero] += 1
        
winrate = {}

for hero in wins.keys():
    winrate[hero] = wins[hero] / games[hero]

pd.DataFrame(winrate.items(), columns = ['hero', 'winrate']).to_csv('processed/winrate.csv', index=False)

Стоит отметить, что у нас получилось одно неожиданное значение `hero_id` - 0. Это значит, что игрок покинул матч, не успев выбрать героя. Винрейт при этом событии, как можно видеть, довольно низкий (сложно играть вчетвером против пятерых). По идее, это стоило бы выкинуть, потому что по смыслу не подходит, но я оставлю, потому что это хороший критерий.

## Сбор признаков

Давайте теперь соберём в одну табличку данные которые планируем использовать.

In [5]:
combined = [{'mid': i} for i in range(match_df.shape[0])]

for _, row in players_df.iterrows():
    mid = row['match_id']
    player = 'player_' + str(row['player_slot'])
    
    combined[mid][player + '_kills'] = row['kills']
    combined[mid][player + '_deaths'] = row['deaths']
    combined[mid][player + '_assists'] = row['assists']
    
    combined[mid][player + '_winrate'] = winrate[row['hero_id']]
    
    combined[mid][player + '_gold'] = row['gold_spent']

Кроме этого, было бы интересно отсортировать игроков одной команды по золоту и записать в признаки в таком ключе (регрессия, которую я хочу позже применить к этому датасету, не очень хорошо умеет брать функцию `max`).

In [6]:
for row in combined:
    radiant_golds = [row[player_gold] for player_gold in ['player_' + str(slot) + '_gold' for slot in radiant]]
    radiant_golds.sort()
    
    for i in range(len(radiant_golds)):
        row['radiant_top_gold_' + str(i)] = radiant_golds[i]
        
    row['radiant_sum_gold'] = sum(radiant_golds)
    
    dire_golds = [row[player_gold] for player_gold in ['player_' + str(slot) + '_gold' for slot in dire]]
    dire_golds.sort()
    
    for i in range(len(dire_golds)):
        row['dire_top_gold_' + str(i)] = dire_golds[i]
        
    row['dire_sum_gold'] = sum(dire_golds)

С винрейтами примерно такая же ситуация, поэтому давайте попробуем добавить ещё один столбец с "винрейтом команды". Это, конечно, будет не совсем справедливый вирейт команды, однако у нас не так уж много данных, чтобы попытаться учесть ещё и синергии героев.

In [7]:
for row in combined:
    radiant_wr = [row[player_wr] for player_wr in ['player_' + str(slot) + '_winrate' for slot in radiant]]
        
    row['radiant_wr'] = np.prod(radiant_wr)
    
    dire_wr = [row[player_wr] for player_wr in ['player_' + str(slot) + '_winrate' for slot in dire]]
        
    row['dire_wr'] = np.prod(dire_wr)

Теперь запишем это в файл:

In [8]:
pd.DataFrame(combined).to_csv('processed/combined.csv', index=False)

## Лейблы

Теперь давайте причешем лейблы для наших данных - предсказывать мы будем победу команды Radiant.

In [9]:
labels = [dict() for i in range(match_df.shape[0])]

for i, row in match_df.iterrows():
    labels[i]['match_id'] = i
    labels[i]['radiant_win'] = row['radiant_win']
    
pd.DataFrame(labels).to_csv('processed/labels.csv', index=False)